In [1]:
import sys
import arcpy
from arcpy import env
import os
import os.path
import arcpy.cartography as CA
arcpy.env.overwriteOutput = True
import pandas as pd
from datetime import datetime

In [2]:
def bufferPA(region_file, region_relevant, PA_file, PA_relevant, PAonly_relevant, PA_relevant_buffer, region_code):
    # Process: Select analysis
#     arcpy.Select_analysis(region_file, region_relevant, "STATEFP = '%s'" % region_code, )
#     arcpy.Select_analysis(region_file, region_relevant, "REG_NUM = %s" % region_code, )
    arcpy.Select_analysis(region_file, region_relevant, "GEOID = '%s'" % region_code, )
    # Process: Clip PA by region
    arcpy.analysis.Clip(PA_file, region_relevant, PA_relevant, "#")
    print('clipped')
    # Process: Select only PAs
    arcpy.Select_analysis(PA_relevant, PAonly_relevant, "ifPA = '1'",)
    print('selected')
    # Process: Buffer
    arcpy.Buffer_analysis(PAonly_relevant, PA_relevant_buffer, "230 kilometers", "FULL", "ROUND", "ALL")

In [3]:
def getRef(PAonly_relevant, PA_relevant_dissolved, PA_relevant_ds_gcs):
    # Process: Dissolve
    arcpy.Dissolve_management(PAonly_relevant, PA_relevant_dissolved)

    gcs = arcpy.SpatialReference("WGS 1984")
    print(gcs)
    # Process: Project to a geographic coordinate system
    print('start projecting')
    arcpy.management.Project(PA_relevant_dissolved, PA_relevant_ds_gcs,gcs,'WGS_1984_(ITRF00)_To_NAD_1983')
    # Process: Calculate geometry field lon and lat
    print('projectiion done')
    arcpy.CalculateGeometryAttributes_management(PA_relevant_ds_gcs, [["lon","CENTROID_X"],
                                                          ["lat","CENTROID_Y"]])
    lon = arcpy.da.SearchCursor(PA_relevant_ds_gcs, ("lon",)).next()[0]
    lat = arcpy.da.SearchCursor(PA_relevant_ds_gcs, ("lat",)).next()[0]
    wkt = "PROJCS['World_Azimuthal_Equidistant',\
           GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],\
           PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],\
           PROJECTION['Azimuthal_Equidistant'],\
           PARAMETER['False_Easting',0.0],\
           PARAMETER['False_Northing',0.0],\
           PARAMETER['Central_Meridian',{0}],\
           PARAMETER['Latitude_Of_Origin',{1}],\
           UNIT['Meter',1.0]]".format(str(lon), str(lat))
    azimuthal_sr = arcpy.SpatialReference(text=wkt)
    return azimuthal_sr

In [4]:
# def getOutput(PA_relevant_buffered_azimuthal, PA_final, outpath, tab_final):
def getOutput(PA_relevant_buffered_azimuthal, PA_final, finaloutgdb, tab_final):
    # Process: Add Field nodeID
    arcpy.AddField_management(PA_relevant_buffered_azimuthal, "nodeID", "LONG", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")

    # Process: Calculate Field
    arcpy.CalculateField_management(PA_relevant_buffered_azimuthal, "nodeID", "!OBJECTID!", "PYTHON", "")
    print("Field nodeID computed")

    # Process: Copy Features (2)
    arcpy.CopyFeatures_management(PA_relevant_buffered_azimuthal, PA_final, "", "0", "0", "0")
    print("Final layer copied")
    
    # Table to table (txt) with field mapping (in alternative to copy rows)
#     arcpy.TableToTable_conversion(in_rows=PA_final, out_path=outpath, out_name=tab_final)
    arcpy.TableToTable_conversion(in_rows=PA_final, out_path=finaloutgdb, out_name=tab_final)
    print("Attribute Table exported")

In [5]:
# Input setting
#--input files of region--

#--states--
# region_folder = r'C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\REGION\Counties\tl_2020_us_county'
# region_file = region_folder + '/tl_2020_us_county2state_proj.shp'
# region_abbr_file = r'C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\REGION\Counties\states_fp_name.csv'
# regionname = 'STATE'
# region_abbr_df = pd.read_csv(region_abbr_file)

#--doi regions--
# region_folder = r'C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\REGION\DOIRegions'
# region_file = region_folder + '/DOI_Regions_NAALBERS.shp'
# region_abbr_file = r'C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\REGION\DOIRegions\DOI_name.csv'
# regionname = 'DOI'
# region_abbr_df = pd.read_csv(region_abbr_file)

#--counties--
region_folder = r'C:\Users\wyang80\Desktop\ProtConn\data\Counties\tl_2020_us_county'
region_file = region_folder + '/tl_2020_us_county_230km_all.shp'
region_abbr_file = r'C:\Users\wyang80\Desktop\ProtConn\data\Counties\county_info.csv'
regionname = 'COUNTIES'
region_abbr_df = pd.read_csv(region_abbr_file, sep = '\t')
region_abbr_df['GEOID'] = region_abbr_df['GEOID'].astype(str)
region_abbr_df['ID'] = region_abbr_df['ID'].astype(str)


#--input file of PA--
# ingdb_fullpath = r"C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\ProtConn_July2021_1.gdb"
# ingdb_fullpath = r"C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\ProtConn_Aug2021.gdb"
ingdb_fullpath = r"C:\Users\wyang80\Desktop\ProtConn\data\ProtConn_COUNTIES_May2022.gdb"
PA_file = ingdb_fullpath + '\PA_mexcanus_final_2205_' + regionname

# Output setting
outgdb_folder = r"C:\Users\wyang80\Desktop\ProtConn\data"

#--states--
# outgdb_name = 'States_PA_1.gdb'
# outgdb_fullpath = r"C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\States_PA_1.gdb"
# outpath = r'C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\REGION_PA\STATES_1'

#--doi regions--
# outgdb_name = 'DOI_PA.gdb'
# outgdb_fullpath = r"C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\DOI_PA.gdb"
# outpath = r'C:\Users\ywx12\Desktop\PhD\RA\ProtectedLand\ProtConn_CALC\data\REGION_PA\DOI'
# finaloutgdb = outpath + '\Region_PA_DOI.gdb'
# finaloutgdbname = 'Region_PA_DOI.gdb'

#--doi regions--
outgdb_name = 'COUNTIES_PA.gdb'
outgdb_fullpath = r"C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb"
outpath = r"C:\Users\wyang80\Desktop\ProtConn\data\REGION_PA\COUNTIES"
finaloutgdb = outpath + '\Region_PA_COUNTIES.gdb'
finaloutgdbname = 'Region_PA_COUNTIES.gdb'

In [6]:
region_abbr_df.columns

Index(['OID_', 'GEOID', 'ID', 'NAME'], dtype='object')

In [7]:
# Process: Create GDB
if arcpy.Exists(finaloutgdb):
    print("outgdb already exists.")
else:
    arcpy.CreateFileGDB_management(outpath, finaloutgdbname)
    print("outgdb created")

outgdb already exists.


In [8]:
if arcpy.Exists(outgdb_fullpath):
    print("outgdb already exists.")
else:
    arcpy.CreateFileGDB_management(outgdb_folder, outgdb_name)
    print("outgdb created")

outgdb already exists.


In [9]:
# Intermediate Files
PA_relevant = outgdb_fullpath + '/PA_relevant'
PAonly_relevant = outgdb_fullpath + '/PAonly_relevant'

PA_relevant_dissolved = outgdb_fullpath + '/PA_relevant_dissolved'
# PA_relevant_pt = outgdb_fullpath + '/PA_relevant_pt'
PA_relevant_ds_gcs = outgdb_fullpath + '/PA_relevant_ds_gcs'

PA_relevant_buffer = outgdb_fullpath + '/PA_relevant_buffer'

region_relevant = outgdb_fullpath + '/region_relevant'
region_relevant_azimuthal = outgdb_fullpath + '/region_relevant_azimuthal'

PA_relevant_buffered = outgdb_fullpath + '/PA_relevant_buffered'
PA_relevant_buffered_azimuthal = outgdb_fullpath + '/PA_relevant_buffered_azimuthal'

In [ ]:
for i in range(len(region_abbr_df)):
# for i in range(4, 5):
    try:
        region_code = str(region_abbr_df.iloc[i][1])
        print(region_code)
        
        # for states, unnecessary for doi regions
#         if len(region_code) == 1:
#             region_code = '0' + region_code
        region_abbr = str(region_abbr_df.iloc[i][2])
        target_region = region_abbr_df.iloc[i][3]
        print('Start working on: ', target_region)
        time0 = time.time()

        for j in [PA_relevant, PAonly_relevant, PA_relevant_dissolved,
              PA_relevant_ds_gcs, PA_relevant_buffer, 
              region_relevant, region_relevant_azimuthal,
              PA_relevant_buffered, PA_relevant_buffered_azimuthal]:
            if arcpy.Exists(j):
                arcpy.Delete_management(j)

        # outfiles
#         PA_final = finaloutgdb + '/PA_230km_' + region_abbr + '_Aug2021'
#         tab_final = 'PA_230km_' + region_abbr + '_Aug2021'
        PA_final =  outpath + '/PA_230km_' + region_abbr + '_May2022.shp'
        tab_final = 'PA_230km_' + region_abbr + '_May2022'
        
        
        if arcpy.Exists(PA_final):
            pass
        else:
            bufferPA(region_file, region_relevant, PA_file, PA_relevant, PAonly_relevant, PA_relevant_buffer, region_code)
            print('buffer done')
            azimuthal_sr = getRef(PAonly_relevant, PA_relevant_dissolved, PA_relevant_ds_gcs)
            print('projection done')
            # Process: Clip
            arcpy.analysis.Clip(PA_file, PA_relevant_buffer, PA_relevant_buffered, "#")
            print('clip done')
            # Process: Project to azimuthal equidistant projection
            arcpy.management.Project(PA_relevant_buffered, PA_relevant_buffered_azimuthal, azimuthal_sr)

            # Process: Repair Geometry
            arcpy.RepairGeometry_management(PA_relevant_buffered_azimuthal, "DELETE_NULL")

            getOutput(PA_relevant_buffered_azimuthal, PA_final, finaloutgdb, tab_final)

            time1 = time.time()
            totaltime = (time1-time0)/60
            print(target_region + ' finished, total time elapsed: ' +  "{:.2f}".format(totaltime))
    except Exception as e:
        print(e)

31039
Start working on:  Cuming
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A47BB70>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

53069
Start working on:  Wahkiakum
35011
Start working on:  De Baca
31109
Start working on:  Lancaster
31129
Start working on:  Nuckolls
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B830B2250>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

72085
Start working on:  Las Piedras
46099
Start working on:  Minnehaha
48327
Start working on:  Menard
clipped
selected
buffer done
<geo

Field nodeID computed
Final layer copied
Attribute Table exported
Walsh finished, total time elapsed: 0.57
39115
Start working on:  Morgan
39133
Start working on:  Portage
56041
Start working on:  Uinta
32013
Start working on:  Humboldt
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1850>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Humboldt finished, total time elapsed: 0.52
48247
Start working on:  Jim Hogg
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184FD0>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

8017
Start working on:  Cheyenne
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0


37109
Start working on:  Lincoln
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1D10>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

5091
Start working on:  Miller
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184330>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

12095
Start working on:  Orange
8111
Start working on:  San Juan
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1270>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\Prot

<geoprocessing spatial reference object object at 0x0000024B8A184F10>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Scioto finished, total time elapsed: 0.52
37115
Start working on:  Madison
16053
Start working on:  Jerome
39163
Start working on:  Vinton
48309
Start working on:  McLennan
16037
Start working on:  Custer
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF13F0>
start projecting
projectiion done
projection done
Failed to execute. Parameters are not valid.
ERROR 000732: Clip Features: Dataset C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_buffer does not exist or is not supported
Failed to execute (Clip).

19089
Start working on:  Howard
12099
Start working on:  Palm Beach
13181
Start working on:  Lincoln
1013
Start working on:  Butler
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184830>
s


51119
Start working on:  Middlesex
42065
Start working on:  Jefferson
18089
Start working on:  Lake
47003
Start working on:  Bedford
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184DF0>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

47027
Start working on:  Clay
17029
Start working on:  Coles
47097
Start working on:  Lauderdale
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1550>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Lauderdale finished, total time elapsed: 0.55
41001
Start working on:  Baker
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184890>
start projecting
proj

...

Attribute Table exported
McKenzie finished, total time elapsed: 0.80
12091
Start working on:  Okaloosa
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF10D0>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

6001
Start working on:  Alameda
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184B10>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

17013
Start working on:  Calhoun
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1AB0>
start projecting
projectiion done
proje

<geoprocessing spatial reference object object at 0x0000024B8ABF1350>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Cherokee finished, total time elapsed: 0.83
5145
Start working on:  White
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184090>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

28083
Start working on:  Leflore
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1890>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Leflore finished, total time elapsed: 0.93
29053
Start working on:  Cooper
clipped
selected
buffer done
<geoprocessing spatia


21057
Start working on:  Cumberland
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1550>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Cumberland finished, total time elapsed: 0.85
28107
Start working on:  Panola
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A1845D0>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Panola finished, total time elapsed: 0.93
48195
Start working on:  Hansford
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1EB0>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

48071
Start 

start projecting
projectiion done
projection done
Failed to execute. Parameters are not valid.
ERROR 000732: Clip Features: Dataset C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_buffer does not exist or is not supported
Failed to execute (Clip).

22109
Start working on:  Terrebonne
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1430>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Terrebonne finished, total time elapsed: 0.83
13155
Start working on:  Irwin
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184BB0>
start projecting
projectiion done
ERROR 000590: C:\Users\wyang80\Desktop\ProtConn\data\COUNTIES_PA.gdb/PA_relevant_ds_gcs is empty.
ERROR 130051: Input feature class is not registered as versioned.
Failed to execute (CalculateGeometryAttributes).

2185
Start working on:  North Slope
clipped
sel


34013
Start working on:  Essex
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A1842F0>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Essex finished, total time elapsed: 0.90
29115
Start working on:  Linn
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8ABF1990>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Linn finished, total time elapsed: 0.85
13209
Start working on:  Montgomery
clipped
selected
buffer done
<geoprocessing spatial reference object object at 0x0000024B8A184FB0>
start projecting
projectiion done
projection done
clip done
Field nodeID computed
Final layer copied
Attribute Table exported
Montgomery finished, total time elapsed: 0.89
41055
Start working on:  Sherman
clipped
selected
buffer done
<geoprocessing spatial reference objec